In [15]:
import shared.config as config
import pymongo
import gridfs
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
#import attributes


import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.types import VARCHAR

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

from sqlalchemy import and_, or_

from PIL import Image, ExifTags
import magic
import io
#import shared.image_util as iu
import os

from sqlalchemy.dialects import postgresql
from sqlalchemy.schema import CreateTable
from sqlalchemy import Table, Column, String, MetaData

#from models import Stelling, Project, Vondst

In [16]:
def getAnalyseCollection():   
    myclient = pymongo.MongoClient(str(config.MONGO_URI))
    analyseDb = myclient[str(config.DB_ANALYSE)]
    return analyseDb[config.COLL_ANALYSE]

def getAnalyseCleanCollection():   
    myclient = pymongo.MongoClient(str(config.MONGO_URI))
    analyseDb = myclient[str(config.DB_ANALYSE)]
    return analyseDb[config.COLL_ANALYSE_CLEAN]

In [17]:
regexTable = re.compile(r'\'table\': \'(.*?)\'') # regex to replace Object
regexProject = re.compile(r'\'projectcd\': \'(.*?)\'') # regex to replace Object
def getTable(brondata):  
    try:
        return regexTable.search(str(brondata)).group(1)
    except:
        return None

def getProject(brondata):    
    try:
        return regexProject.search(str(brondata)).group(1)
    except:
        return None

def firstValue(*kargs):
    for x in kargs:
        if x and not_empty(x):
            return x
    return None

In [20]:
#Aggregate Pipeline
collection = getAnalyseCollection()
cleancollection = getAnalyseCleanCollection()

# get all Photo records
df_foto = pd.DataFrame(list(collection.find({"soort": "Foto"})))

# get all Photo descr records
df_fotobeschr = pd.DataFrame(list(collection.find({"soort": "Fotobeschrijving"})))
df_fotobeschr["abcd-nr"] = df_fotobeschr.apply(lambda x: x["pad"].split('\\')[-1].lower() if x["pad"] and type(x["pad"]) == str else '', axis=1) 

df_fotokoppel = pd.DataFrame(list(collection.find({"soort": "Fotokoppel"})))
df_fotokoppel["abcd-nr"] = df_fotokoppel.apply(lambda x: x["abcd-nr"].lower() if x["abcd-nr"] and type(x["abcd-nr"]) == str else '', axis=1) 

# Merge dataframes to get a complete Photo-dataframe
df_merge = df_foto.merge(df_fotokoppel, how="left", right_on="bestandsnaam", left_on="fileName", suffixes=("", "_koppel"))
df_merge = df_merge.merge(df_fotobeschr, how="left", on=["abcd-nr", "projectcd"], suffixes=("", "_beschr"))
df_merge['soort'] = 'Foto'
df_merge['brondata'] = df_merge.apply(lambda x: [x['brondata'], x['brondata_beschr']], axis=1)
df_merge['wasstraat'] = df_merge.apply(lambda x: [{'projectcd': x['projectcd'], 'table': getTable(elem)} for elem in x['brondata'] if getTable(elem)], axis=1)  
df_merge['materiaal'] = df_merge.apply(lambda x: firstValue(x['materiaal'], x['materiaalgroep']) if 'materiaal' in df_merge.columns else x['materiaalgroep'],axis=1)
df_merge = df_merge[['_id', 'fileName', 'imageUUID', 'imageMiddleUUID', 'imageThumbUUID',
    'fileType', 'directory', 'mime_type', 'fototype', 'soort', 'projectcd',
    'materiaal', 'putnr', 'vondstnr', 'fotonr', 'vondstkey_met_putnr',
    'key', 'key_project', 'key_project_type', 'key_vondst',
    'key_artefact', 'subnr', 'brondata',
    'pad', 'spoornr', 'profiel', 'subnr', 'datum', 'omschrijving', 'vlaknr', 'richting', 'wasstraat', 'fotosoort']]


#updates= [ pymongo.ReplaceOne({"_id": record['_id']}, record, upsert=True) for record in [v.dropna().to_dict() for k,v in df_merge.iterrows()]]  # 
#if len(updates) > 0:
#    logger.info(f"Upserting {len(updates)} photo records.")
#    result = cleancollection.bulk_write(updates)
#else:
#    logger.warning("Could not merge photo data due to empty dataset")

collection.database.client.close()
cleancollection.database.client.close()


df_merge

NameError: name 'not_empty' is not defined

In [29]:
doc = {}
doc['tekeningcd'] = 'B00001'

doc['tekeningcd'] = str(doc['tekeningcd']).replace('?', '').strip()
matchObj = re.match( r'^([A-Z])([0-9]+)$', doc['tekeningcd'], re.M|re.I)
if matchObj:
    doc['tekeningcd'] = matchObj.group(1) + str(int(matchObj.group(2))).zfill(3)

doc['tekeningcd']

'B001'

In [23]:
AGGREGATE_MERGE = [{'$match': {'soort': 'XXX'}},
 {'$group': {
    '_id': {'$mergeObjects': '$$ROOT'},
    'brondata': {'$addToSet': '$$ROOT.brondata'},
    'wasstraat': {'$addToSet': {'projectcd': '$$ROOT.brondata.projectcd', 'table': '$$ROOT.brondata.table'}}}},
 {'$project': {'_id.brondata': 0}},
 {'$addFields': {'_id.brondata': '$brondata',
    'doc.wasstraat': '$wasstraat'}},
 {'$replaceRoot': {'newRoot': '$_id'}},
 {'$addFields': {'brondata_count': {'$size': '$brondata'}}},
 { "$merge": { "into": { "db": config.DB_ANALYSE, "coll": config.COLL_ANALYSE_CLEAN }, "on": "_id",  "whenMatched": "replace", "whenNotMatched": "insert" } }
]



AGGREGATE_MERGE.insert(-1, {'$match': {'imageUUID': {'$exists': True}}})
AGGREGATE_MERGE.insert(-1, {'$addFields': {'soort': 'Foto'}})


AGGREGATE_MERGE[0]['$match']['soort'] = 'Tekening'
AGGREGATE_MERGE.pop(-1)
AGGREGATE_MERGE

[{'$match': {'soort': 'Tekening'}},
 {'$group': {'_id': {'$mergeObjects': '$$ROOT'},
   'brondata': {'$addToSet': '$$ROOT.brondata'},
   'wasstraat': {'$addToSet': {'projectcd': '$$ROOT.brondata.projectcd',
     'table': '$$ROOT.brondata.table'}}}},
 {'$project': {'_id.brondata': 0}},
 {'$addFields': {'_id.brondata': '$brondata', 'doc.wasstraat': '$wasstraat'}},
 {'$replaceRoot': {'newRoot': '$_id'}},
 {'$addFields': {'brondata_count': {'$size': '$brondata'}}},
 {'$addFields': {'soort': 'Foto'}}]

In [25]:
collection = getAnalyseCollection()

df = pd.DataFrame(list(collection.aggregate(AGGREGATE_MERGE)))
df[df.key == 'PDB034TB003']

,_id,fileName,fullFileName,imageUUID,imageMiddleUUID,imageThumbUUID,fileType,directory,mime_type,projectcd,...,coupe,datum,details,microfilm,omschrijving,profiel,schaal,volgnr,key_put,periode
2615,630cdd1d75be1aea4d5a9a26,DB034_B003_LZW.tif,/input/images/DB034_Koningsveld IV/D Velddocum...,630cdd1d75be1aea4d5a99f6,630cdd1d75be1aea4d5a9a22,630cdd1d75be1aea4d5a9a24,.tif,/input/images/DB034_Koningsveld IV/D Velddocum...,image/jpeg,DB034,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4681,630cdcaca4d57c803c4051a3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB034,...,NaN,1970-01-01 00:00:00.016,skeletten,NaN,skeletten put 10; aan de onderkant overzicht v...,NaN,20,NaN,PDB034P9,NaN
